# This Notebook presents an expriment of Theory Of Mind on Schelling Coordination Game

In [3]:
# Import libraries.
using Turing, StatsPlots, Random

LoadError: ArgumentError: Package Turing not found in current path:
- Run `import Pkg; Pkg.add("Turing")` to install the Turing package.


In [5]:
import Pkg; Pkg.add("Turing")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%.1%
  Resolving package versions...
  Installed CommonSolve ──────────────── v0.2.0
  Installed ConstructionBase ─────────── v1.0.0
  Installed ScientificTypes ──────────── v1.1.1
  Installed MappedArrays ─────────────── v0.3.0
  Installed CompositionsBase ─────────── v0.1.0
  Installed Libtask ──────────────────── v0.5.1
src/bijectors/simplex.jl: truncated gzip input
tar: Error exit delayed from previous errors.
┌ Warning: failed to extract archive downloaded from https://pkg.julialang.org/package/76274a88-744f-5084-9051-94815aaf08c4/fe3cf20173d8bd9a65e6c18f20d2efee5fa4e83e
└ @ Pkg.Operations /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Operations.jl:559
  Installed InitialValues ────────────── v0.2.10
  Installed LeftChildRightSiblingTrees ─ v0.1.2
  Installed Hwloc_jll ────────────────── v2.2.0+0
  Installed S

We have 2 players: Bob and Alice.
They Want to arrive to the same point. they can't communicate, they only can infernce about each other.
We will model it using PPL. And use Practicle Gibbs Sampling.

In [2]:
@model function prior_location(popularity_of_place = 0.9)
    catagorical_dist = zeros(2)
    coin ~ Bernoulli(popularity_of_place)
    catagorical_dist[coin + 1] = 1
    location ~ Categorical(catagorical_dist)
    return location
end

prior_location (generic function with 2 methods)

## Location prior

```Javascript
var locationPrior = function() {
  if (flip(.55)) {
    return 'popular-bar';
  } else {
    return 'unpopular-bar';
  }
};
```

In [2]:
@model function locationPrior(popularity=0.5)
    popular_bar ~ Bernoulli(popularity)
    return popular_bar
end

LoadError: LoadError: UndefVarError: @model not defined
in expression starting at In[2]:1

```Javascript
var bob = function() {
  return Infer({ model() {
    var myLocation = locationPrior();
    return myLocation;
  }});
};
```

In [3]:
@model function bob(depth = 0, popularity=0.55)
    my_location = locationPrior(popularity)
    return my_location
end

bob (generic function with 3 methods)

In [4]:
function get_location_2(samples)
    return round(Int64, mean(samples[:my_location]))
end

get_location_2 (generic function with 1 method)

In [5]:
function get_location(samples)
    return samples[:my_location][end]
end

get_location (generic function with 1 method)

```Javascript
var alice = function() {
  return Infer({ model() {
    var myLocation = locationPrior();
    var bobLocation = sample(bob());
    condition(myLocation === bobLocation);
    return myLocation;
  }});
};

```

In [6]:
@model function alice(other_person_place, popularity = 0.55, depth = 0)
    my_location = locationPrior(popularity)
    other_location = get_location(sample(other_person_place(), PG(10), 10))
    other_location ~ Dirac(my_location)
    return my_location
end

alice (generic function with 3 methods)

In [7]:
samples = sample(alice(bob), PG(10), 10)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (10×4×1 Array{Float64,3}):

Log evidence      = 0.0
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = my_location, other_location
internals         = logevidence, lp

Summary Statistics
      parameters      mean       std   naive_se      mcse       ess      rhat 
          Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

     my_location    0.6000    0.5164     0.1633   missing    9.6552    1.0583
  other_location    0.6000    0.5164     0.1633   missing    9.6552    1.0583

Quantiles
      parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
          Symbol   Float64   Float64   Float64   Float64   Float64 

     my_location    0.0000    0.0000    1.0000    1.0000    1.0000
  other_location    0.0000    0.0000    1.0000    1.0000    1.0000


In [8]:
@model function alice_2(other_person_place, popularity = 0.55, depth = 0)
    my_location ~ Bernoulli(popularity)
    other_location = get_location(sample(other_person_place(alice_2, popularity, depth -1), PG(5), 5))
    other_location ~ Dirac(my_location)
    return my_location
end

alice_2 (generic function with 3 methods)

In [14]:
@model function bob_2(other_person_place, popularity=0.55, depth = 0)
    my_location ~ Bernoulli(popularity)
    if depth == 0
       return my_location 
    else
        other_location = get_location(sample(other_person_place(bob_2, popularity, depth), PG(10), 10))
        other_location ~ Dirac(my_location)
        return my_location
    end
end

bob_2 (generic function with 3 methods)

In [ ]:
samples = sample(alice_2(bob_2, 0.55, 2), PG(10), 20)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 